In [2]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [4]:
# Create Path
path = r'/Users/nhungphamthi/Documents/data analytis/Achievement 4 Project'

In [10]:
# Import ords_prods_merge dataframe ( Before importing, I deleted  a few columns, resized and changed the name of column "order_dow" to "orders_day_of_the_week"
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_resized.pkl'))

# 1.1 Create "price_label" column

In [14]:
# Define a function for "price label"
def price_label(row): 
    if row['prices'] <= 5: return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15): return 'Mid-range product'
    elif row['prices'] > 15:  return 'High range product'
    else: return 'Not enough data'

In [16]:
df_ords_prods_merge.apply(price_label, axis=1)

0           Mid-range product
1           Mid-range product
2           Low-range product
3           Low-range product
4           Low-range product
                  ...        
32404854    Mid-range product
32404855    Mid-range product
32404856    Low-range product
32404857    Low-range product
32404858    Low-range product
Length: 32404859, dtype: object

In [23]:
# Create a subset of first million rows
df = df_ords_prods_merge[:1000000]

In [27]:
# Use loc() method on df subset to apply price range function.
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [51]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [29]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [31]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     673340
Low-range product     314392
High-range product     12268
Name: count, dtype: int64

In [37]:
# Use loc() method on entire ords_prods_merge dataframe to apply price range function.
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [39]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [47]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                1000000 non-null  int32  
 1   user_id                 1000000 non-null  int32  
 2   order_number            1000000 non-null  int8   
 3   orders_day_of_the_week  1000000 non-null  int8   
 4   order_hour_of_day       1000000 non-null  int8   
 5   days_since_prior_order  936384 non-null   float32
 6   new_customer            1000000 non-null  bool   
 7   add_to_cart_order       1000000 non-null  int32  
 8   reordered               1000000 non-null  int8   
 9   prices                  1000000 non-null  float32
 10  price_range_loc         1000000 non-null  object 
dtypes: bool(1), float32(2), int32(3), int8(4), object(1)
memory usage: 31.5+ MB


In [41]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [53]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21865987
Low-range product     10126321
High-range product      412551
Name: count, dtype: int64

# 1.2. Create "busiest_day" column

In [61]:
# Find frequency of orders_day_of_the_week.
df_ords_prods_merge['orders_day_of_the_week'].value_counts(dropna = False)

orders_day_of_the_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [65]:
# Use for-loop to determine whether orders are on "busiest day" (0 = Saturday), "least busy" (4 = Wednesday), or "regularly busy" (other days of the week).
result = []
for value in df_ords_prods_merge["orders_day_of_the_week"]:
  if value == 0: result.append("Busiest day")
  elif value == 4: result.append("Least busy")
  else: result.append("Regularly busy")

In [67]:
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy

In [69]:
# Create new column in ords_prods_merge to view results in context.
df_ords_prods_merge['busiest_day'] = result

In [71]:
df_ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

# 2. Update "busiest day" column to "busiest days" to identify order day of the week as "Busiest days," "Slowest days," or "Regularly busy."

In [74]:
# Create column “busiest_days" from "orders_day_of_week" to identify two busiest days (0, 1), slowest days (4, 3), or regularly busy (all other days).

result_2 = []
for value in df_ords_prods_merge["orders_day_of_the_week"]:
  if value == 0 or value == 1:  result_2.append("Busiest days")
  elif value == 4 or value == 3: result_2.append("Slowest days")
  else: result_2.append("Regularly busy")

In [76]:
result_2

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest day

# 3. Check values of new "busiest days" column for accuracy with observations in markdown format.

In [82]:
# Create new 'busiest_days' column in ords_prods_merge to view results in context.
df_ords_prods_merge['busiest_days'] = result_2

In [84]:
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    12916111
Busiest days      11864412
Slowest days       7624336
Name: count, dtype: int64

In [92]:
# Check output of ords_prods_merge with new "busiest days" column.
df_ords_prods_merge.head ()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,new_customer,add_to_cart_order,reordered,prices,price_range_loc,busiest_day,busiest_days
0,2539329,1,1,2,8,NaN,True,1,0,9.0,Mid-range product,Regularly busy,Regularly busy
1,2539329,1,1,2,8,NaN,True,2,0,12.5,Mid-range product,Regularly busy,Regularly busy
2,2539329,1,1,2,8,NaN,True,3,0,4.4,Low-range product,Regularly busy,Regularly busy
3,2539329,1,1,2,8,NaN,True,4,0,4.7,Low-range product,Regularly busy,Regularly busy
4,2539329,1,1,2,8,NaN,True,5,0,1.0,Low-range product,Regularly busy,Regularly busy


The total value counts for the ords_prods_merge dataframe is equal to the total sum of all groupings (i.e. "Regularly busy" + "Busiest days" + "Slowest days"). In addition, the listed sums for each label match the sum totals for the qualifying days of the week (e.g. Total "Busiest days" = 11864412, which matches orders_day_of_week 0 + 1, or 6204182 + 5660230).

# 4. Create new column "busiest_period_of_day" to identify time periods “Most orders,” “Average orders,” and “Fewest orders.”

In [94]:
# Check value counts in "order_hour_of_day" column.
df_ords_prods_merge['order_hour_of_day'].value_counts()

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

The value counts listed above will be split into equal thirds for the following labels: "Most orders" = 10, 11, 14, 15, 13, 12, 16, and 9; "Fewest orders" = 23, 6, 0, 1, 5, 2, 4, and 3; "Average orders" = all remaining values in order_hour_of_day.

In [97]:
# Create for-loop if statement labeling periods of time as “Most orders,” “Average orders,” and “Fewest orders.”
result_3 = []
for value in df_ords_prods_merge["order_hour_of_day"]:
  if value in [10, 11, 14, 15, 13, 12, 16, 9]: result_3.append("Most orders")
  elif value in [23, 6, 0, 1, 5, 2, 4, 3]: result_3.append("Fewest orders")
  else: result_3.append("Average orders")

In [99]:
result_3

['Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Avera

In [101]:
# Create new column "busiest_period_of_day" in ords_prods_merge.
df_ords_prods_merge['busiest_period_of_day'] = result_3

# 5. Print frequency of "busiest_period_of_day" column

In [104]:
# Print value counts in "busiest_period_of_day" column.
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21118071
Average orders     9997651
Fewest orders      1289137
Name: count, dtype: int64

In [112]:
df_ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int32  
 1   user_id                 int32  
 2   order_number            int8   
 3   orders_day_of_the_week  int8   
 4   order_hour_of_day       int8   
 5   days_since_prior_order  float32
 6   new_customer            bool   
 7   add_to_cart_order       int32  
 8   reordered               int8   
 9   prices                  float32
 10  price_range_loc         object 
 11  busiest_day             object 
 12  busiest_days            object 
 13  busiest_period_of_day   object 
dtypes: bool(1), float32(2), int32(3), int8(4), object(4)
memory usage: 1.7+ GB


# Step 7. Export dataframe as a pickle file to “Prepared Data” folder

In [107]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))